# Práctica de Bioinformática: Análisis Estructural de Proteínas

* Asignatura: Bioinformática  
* Laboratorio: Sesión 06 - Estructuras Tridimensionales  

---

### Integrantes del equipo:
* Adrián Ojeda
* Raúl Mendoza

---

### Introducción
En este jupyter notebook utilizaremos la librería Biopython para analizar estructuras tridimensionales de proteínas. Se abordarán tareas fundamentales como:
1.  Cálculo de distancias atómicas y ángulos diedros.
2.  Determinación del centro de masas.
3.  Alineamiento de secuencias y estructuras.
4.  Análisis filogenético básico.

Para ello, trabajaremos con proteínas reales obtenidas del Protein Data Bank (PDB), como la Hemoglobina y la Lisozima, así como neuropéptidos (Hipocretinas/Orexinas).

In [33]:
# Instalación de librerías necesarias (si no tienes instalados la librerías necesarias en tu entorno, descomentar la siguiente línea)
# !pip install biopython numpy

import sys
import numpy as np
import warnings

from Bio.PDB import PDBList, MMCIFParser
from Bio.PDB.vectors import calc_dihedral, Vector

warnings.filterwarnings('ignore')

print(f"Librerías importadas correctamente. Versión de Python: {sys.version.split()[0]}")

Librerías importadas correctamente. Versión de Python: 3.13.1


## Ejercicio 1: La Hemoglobina Humana (1A3N)

* Objetivo: Descargar la estructura de la proteína de la hemoglobina humana (código PDB: 1A3N), visualizarla conceptualmente mediante código y realizar cálculos geométricos.

La hemoglobina es una hemoproteína de la sangre que transporta oxígeno. En este ejercicio realizaremos las siguientes operaciones:

1.  (a) Calcular la **distancia euclidiana** entre los átomos de Oxígeno (O) del primer y último residuo de la cadena A.
    * Fórmula de distancia: $d(p, q) = \sqrt{(q_1 - p_1)^2 + (q_2 - p_2)^2 + (q_3 - p_3)^2}$
2.  (b) Calcular el **ángulo diedro** ($\phi$ o $\psi$) formado por los átomos N, CA, C y O del primer residuo de la cadena A.
3.  (c) Calcular el **centro de masas** ($R$) de toda la estructura.
    * $R = \frac{1}{M} \sum_{i=1}^{n} m_i \mathbf{r}_i$

In [34]:
pdb_id = "1A3N"
pdbl = PDBList()
file_path = pdbl.retrieve_pdb_file(pdb_id, pdir='.', file_format='mmCif')
parser = MMCIFParser()
structure = parser.get_structure(pdb_id, file_path)

print(f"Estructura {pdb_id} cargada exitosamente.")

Structure exists: '.\1a3n.cif' 
Estructura 1A3N cargada exitosamente.


In [35]:
model = structure[0]
chain_a = model['A']
residues = [res for res in chain_a if res.id[0] == ' ']
first_res = residues[0]
last_res = residues[-1]

print(f"- Apartado (a): Distancia")
print(f"Primer residuo: {first_res.get_resname()} (ID: {first_res.id[1]})")
print(f"Último residuo: {last_res.get_resname()} (ID: {last_res.id[1]})")

if 'O' in first_res and 'O' in last_res:
    atom_o_first = first_res['O']
    atom_o_last = last_res['O']
    
    distance = atom_o_first - atom_o_last
    print(f"Distancia entre átomos O: {distance:.4f} Å")
else:
    print("Error: No se encontraron átomos de oxígeno en los residuos extremos.")

- Apartado (a): Distancia
Primer residuo: VAL (ID: 1)
Último residuo: ARG (ID: 141)
Distancia entre átomos O: 22.2284 Å


In [36]:
print(f"- Apartado (b): Ángulo Diedro")
try:
    n = first_res['N']
    ca = first_res['CA']
    c = first_res['C']
    o = first_res['O']
    
    vector_n = n.get_vector()
    vector_ca = ca.get_vector()
    vector_c = c.get_vector()
    vector_o = o.get_vector()
    
    angle_rad = calc_dihedral(vector_n, vector_ca, vector_c, vector_o)
    angle_deg = np.degrees(angle_rad)

    print(f"Átomes usados: N, CA, C, O de {first_res.get_resname()}")
    print(f"Ángulo diedro: {angle_deg:.4f} grados")
    
except KeyError as e:
    print(f"No se pudo calcular el diedro, falta el átomo: {e}")

- Apartado (b): Ángulo Diedro
Átomes usados: N, CA, C, O de VAL
Ángulo diedro: -25.6655 grados


In [37]:
print(f"- Apartado (c): Centro de Masas")

def calculate_center_of_mass(entity):
    atomic_masses = {"H": 1.008, "C": 12.011, "N": 14.007, "O": 15.999, "S": 32.065, "P": 30.974, "FE": 55.845}
    total_mass = 0.0
    center_of_mass = np.array([0.0, 0.0, 0.0])
    
    for atom in entity.get_atoms():
        element = atom.element.upper().strip()
        mass = atomic_masses.get(element, 12.01) 
        
        pos = atom.get_coord()
        center_of_mass += pos * mass
        total_mass += mass
        
    return center_of_mass / total_mass

com = calculate_center_of_mass(structure)
print(f"Centro de masas (x, y, z):")
print(f"[{com[0]:.4f}, {com[1]:.4f}, {com[2]:.4f}]")

- Apartado (c): Centro de Masas
Centro de masas (x, y, z):
[14.4469, 2.0080, 13.1803]


## Ejercicio 2: Lisozima del Huevo de Gallina (1LYZ)

* Objetivo: Descargar y analizar la estructura de la Lisozima (código PDB: 1LYZ), una enzima que daña las paredes celulares bacterianas catalizando la hidrólisis de uniones glicosídicas.

En este ejercicio manipularemos listas de átomos directamente para realizar los siguientes cálculos:

1.  (a) **Conteo e identificación:** Determinar el número total de átomos en la estructura e identificar por nombre el primero y el último de la lista plana de átomos.
2.  (b) **Cálculo de ángulo de enlace:** Calcular el ángulo ($\theta$) formado por los tres primeros átomos de la estructura.
    * El ángulo entre tres puntos $A, B, C$ (vectores $\vec{BA}$ y $\vec{BC}$) se calcula mediante el producto escalar:
    * $\theta = \arccos \left( \frac{\vec{BA} \cdot \vec{BC}}{|\vec{BA}| |\vec{BC}|} \right)$
3.  (c) **Localización estructural:** Identificar la cadena y el residuo al que pertenece el átomo situado exactamente en la mitad de la lista de átomos.

In [38]:
from Bio.PDB.vectors import calc_angle
from Bio.PDB import PDBParser

pdb_id_2 = "1LYZ"
file_path_2 = pdbl.retrieve_pdb_file(pdb_id_2, pdir='.', file_format='pdb')
parser_pdb = PDBParser()
structure_lyz = parser_pdb.get_structure(pdb_id_2, file_path_2)

print(f"Estructura {pdb_id_2} cargada exitosamente.")


Structure exists: '.\pdb1lyz.ent' 
Estructura 1LYZ cargada exitosamente.


In [39]:
atoms_list = list(structure_lyz.get_atoms())
num_atoms = len(atoms_list)

first_atom = atoms_list[0]
last_atom = atoms_list[-1]

print(f"- Apartado (a): Estadísticas de Átomos")
print(f"Número total de átomos: {num_atoms}")
print(f"Primer átomo: {first_atom.get_name()} (en residuo {first_atom.get_parent().get_resname()} {first_atom.get_parent().id[1]})")
print(f"Último átomo: {last_atom.get_name()} (en residuo {last_atom.get_parent().get_resname()} {last_atom.get_parent().id[1]})")

- Apartado (a): Estadísticas de Átomos
Número total de átomos: 1102
Primer átomo: N (en residuo LYS 1)
Último átomo: O (en residuo HOH 230)


In [40]:
print(f"\n- Apartado (b): Ángulo entre los 3 primeros átomos")

if num_atoms >= 3:
    a1, a2, a3 = atoms_list[0], atoms_list[1], atoms_list[2]
    v1, v2, v3 = a1.get_vector(), a2.get_vector(), a3.get_vector()
    angle_rad = calc_angle(v1, v2, v3)
    angle_deg = np.degrees(angle_rad)
    
    print(f"Átomos implicados: {a1.get_name()} - {a2.get_name()} - {a3.get_name()}")
    print(f"Ángulo calculado: {angle_deg:.4f} grados")
else:
    print("Error: No hay suficientes átomos en la estructura.")


- Apartado (b): Ángulo entre los 3 primeros átomos
Átomos implicados: N - CA - C
Ángulo calculado: 118.3604 grados


In [41]:
print(f"- Apartado (c): Átomo central ")

mid_index = num_atoms // 2
mid_atom = atoms_list[mid_index]
mid_residue = mid_atom.get_parent()
mid_chain = mid_residue.get_parent()

print(f"Índice medio: {mid_index}")
print(f"Átomo seleccionado: {mid_atom.get_name()}")
print(f"Información de ubicación:")
print(f"  -> Cadena: {mid_chain.id}")
print(f"  -> Residuo: {mid_residue.get_resname()} (ID: {mid_residue.id[1]})")

- Apartado (c): Átomo central 
Índice medio: 551
Átomo seleccionado: N
Información de ubicación:
  -> Cadena: A
  -> Residuo: PRO (ID: 70)


## Ejercicio 3: Hipocretinas / Orexinas

**Introducción:** Las hipocretinas (o orexinas) son neuropéptidos hipotalámicos con un papel crucial en la regulación del ciclo sueño-vigilia, la termorregulación y la conducta alimentaria. Su déficit está asociado a la narcolepsia.

**Objetivos:**
1.  **Obtención de datos:** Descargar las secuencias de la Orexina-A (PDB: 1WSO) y Orexina-B (PDB: 1CQ0).
2.  **Alineamiento de secuencias:** Realizar un alineamiento global entre ambas para determinar su porcentaje de identidad/similitud.
3.  **Búsqueda de homólogos (BLAST):** Utilizar la herramienta BLASTp contra la base de datos non-reduntant para encontrar proteínas evolutivamente relacionadas.
4.  **Filogenia:** Construir un árbol filogenético utilizando el método Neighbor Joining a partir de los resultados del BLAST.
5.  **Análisis Estructural Avanzado:** Desarrollar una función que calcule la distancia máxima intra-molecular y determinar cuál de las dos orexinas está más extendida espacialmente.

In [42]:
from Bio.SeqUtils import seq1
from Bio.Align import PairwiseAligner
from Bio.Blast import NCBIWWW, NCBIXML
from Bio import Phylo
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
from Bio.Align import MultipleSeqAlignment
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.PDB import PDBParser, PDBList
import os
import numpy as np

In [43]:
pdb_ids = ["1WSO", "1CQ0"]
structures = {}
pdbl = PDBList()

print("1. Descargando estructuras y extrayendo secuencias...")
for pid in pdb_ids:
    path = pdbl.retrieve_pdb_file(pid, pdir='.', file_format='pdb')
    structures[pid] = PDBParser(QUIET=True).get_structure(pid, path)

def get_sequence_from_pdb(structure):
    first_chain = next(structure[0].get_chains())
    residues = [r for r in first_chain if r.id[0] == ' ']
    seq_str = "".join([seq1(r.get_resname()) for r in residues])
    return seq_str

seq_a = get_sequence_from_pdb(structures["1WSO"])
seq_b = get_sequence_from_pdb(structures["1CQ0"])

print(f"   > Secuencia Orexina-A (1WSO): {seq_a}")
print(f"   > Secuencia Orexina-B (1CQ0): {seq_b}")

1. Descargando estructuras y extrayendo secuencias...
Structure exists: '.\pdb1wso.ent' 
Structure exists: '.\pdb1cq0.ent' 
   > Secuencia Orexina-A (1WSO): PLPDCCRQKTCSCRLYELLHGAGNHAAGILTL
   > Secuencia Orexina-B (1CQ0): FSGPPGLQGRLQRLLQASGNHAAGILTM


In [44]:
print("2. Realizando alineamiento global...")
aligner = PairwiseAligner()
aligner.mode = 'global'
alignment = aligner.align(seq_a, seq_b)[0]
print(alignment)
print(f"   > Score: {alignment.score}")

2. Realizando alineamiento global...
target            0 PLPDCCRQKTCSCRLYELLHGAGNHAAGILTL 32
                  0 .......|.----||..||...|||||||||. 32
query             0 FSGPPGLQG----RLQRLLQASGNHAAGILTM 28

   > Score: 10.0


In [45]:
print("3. Ejecutando BLASTp para Orexina-A...")
xml_file = "blast_orexin.xml"

if not os.path.exists(xml_file):
    try:
        print("   (Conectando con NCBI, por favor espera...)")
        result_handle = NCBIWWW.qblast("blastp", "nr", seq_a, hitlist_size=15)
        with open(xml_file, "w") as out_handle:
            out_handle.write(result_handle.read())
        result_handle.close()
        print("   > BLAST completado y resultados guardados.")
    except Exception as e:
        print(f"   > Error en la conexión BLAST: {e}")
else:
    print("   > Archivo XML previo encontrado. Usando datos locales.")

3. Ejecutando BLASTp para Orexina-A...
   > Archivo XML previo encontrado. Usando datos locales.


In [46]:
print("4. Construyendo árbol filogenético...")

if os.path.exists(xml_file):
    with open(xml_file) as result_handle:
        blast_record = NCBIXML.read(result_handle)

    seq_records = []
    seq_records.append(SeqRecord(Seq(seq_a), id="Query_1WSO"))

    for i, alignment in enumerate(blast_record.alignments[:10]):
        hsp = alignment.hsps[0]
        raw_id = alignment.hit_id.split('|')[-1][:10]
        unique_id = f"{raw_id}_{i+1}"
        seq_records.append(SeqRecord(Seq(hsp.sbjct), id=unique_id))

    if len(seq_records) > 1:
        min_len = min(len(s) for s in seq_records)
        trimmed_recs = [s[:min_len] for s in seq_records]
        msa = MultipleSeqAlignment(trimmed_recs)

        calculator = DistanceCalculator('identity')
        dm = calculator.get_distance(msa)

        constructor = DistanceTreeConstructor()
        tree = constructor.nj(dm)

        Phylo.draw_ascii(tree)
    else:
        print("   > No hay suficientes secuencias para el árbol.")
else:
    print("   > No se pudo generar el árbol (falta archivo XML).")

4. Construyendo árbol filogenético...
                                                           _______ Query_1WSO
                                                    ______|
                                             ______|      |_______ _1
                                            |      |
                                      ______|      |______ _2
                                     |      |
                              _______|      |______ _3
                             |       |
                       ______|       |______ _4
                      |      |
                ______|      |_______ A_5
               |      |
         ______|      |______ _6
        |      |
  ______|      |______ _7
 |      |
 |      |______ _8
_|
 |______ A_9
 |
 |______ _10



In [47]:
print("5. Análisis Estructural: Distancia máxima intra-molecular")

def calcular_distancia_maxima(structure):
    atoms = list(structure.get_atoms())
    coords = np.array([atom.get_coord() for atom in atoms])

    diff = coords[:, np.newaxis, :] - coords[np.newaxis, :, :]
    dists = np.linalg.norm(diff, axis=-1)
    
    return np.max(dists)

max_a = calcular_distancia_maxima(structures["1WSO"])
max_b = calcular_distancia_maxima(structures["1CQ0"])

print(f"   > Diámetro Orexina-A (1WSO): {max_a:.2f} Å")
print(f"   > Diámetro Orexina-B (1CQ0): {max_b:.2f} Å")

winner = "Orexina-A (1WSO)" if max_a > max_b else "Orexina-B (1CQ0)"
print(f"   >> La estructura más extendida es: {winner}")

5. Análisis Estructural: Distancia máxima intra-molecular
   > Diámetro Orexina-A (1WSO): 44.66 Å
   > Diámetro Orexina-B (1CQ0): 34.73 Å
   >> La estructura más extendida es: Orexina-A (1WSO)
